In [14]:
import os
import numpy as np
from keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# WITH PREPROCESSING

In [6]:
ds_train2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 1250 files belonging to 25 classes.
Using 1000 files for training.
Found 1250 files belonging to 25 classes.
Using 250 files for validation.


In [7]:
class_names = ds_train2.class_names
num_classes = len(ds_train2.class_names)

In [8]:
# preprocessing
preprocess_input = tf.keras.applications.resnet50.preprocess_input

ds_train2_preprocessed = ds_train2.map(lambda x, y: (preprocess_input(x), y))
ds_test2_preprocessed = ds_test2.map(lambda x, y: (preprocess_input(x), y))


In [16]:


# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

# x_3 = Dense(num_classes, activation='relu')(x_2)
x_3 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_2)
x_3 = Dropout(0.5)(x_3)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model2 = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model2.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé

history = model2.fit(ds_train2_preprocessed, epochs=10, callbacks=callbacks, validation_data=ds_test2_preprocessed)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - loss: 10.7411 - sparse_categorical_accuracy: 0.1501 - val_loss: 7.4827 - val_sparse_categorical_accuracy: 0.7040
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 7.0280 - sparse_categorical_accuracy: 0.6165 - val_loss: 5.3255 - val_sparse_categorical_accuracy: 0.7920
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 4.9476 - sparse_categorical_accuracy: 0.7728 - val_loss: 4.0346 - val_sparse_categorical_accuracy: 0.8200
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 3.7401 - sparse_categorical_accuracy: 0.8440 - val_loss: 3.2408 - val_sparse_categorical_accuracy: 0.8200
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 2.9821 - sparse_categorical_accuracy: 0.8652 - val_loss: 2.6582 - val_sparse_categorical_accuracy: 0.8680
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 2.4063 - sparse_categorical_accuracy: 0.8940 - val_loss: 2.2276 - val_sparse_categorical_accuracy: 0.8600
Epoch 7/10
16/1

In [10]:
for image_batch, labels_batch in ds_train2_preprocessed:
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  print(class_pred)
  print(labels_batch)
  break

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step
[ 2 18 14  5 22  5 11  5  6 16  2 17  3  0  6 11  0 11 21 16  6  4  1 18
 17 19 20  5 21 10 18  9 14 14 11  6  4 18 15  3 12 18 15  4 11 22 16  9
  5  7 13  1  6 18  6  8  6 22  6  7 10  7 20  3]
tf.Tensor(
[ 2 18 14  6 22  5 11  5  6 16  2 17  3  0  6 11  0 11 21 16  6  4  1 18
 17 19 20  5 21 10 18  9 14 14 11  6  4 18 15  3 12 18 15  4 11 22 16  9
  5  7 13  1  6 18  6  8  6 22  6  7 10  8 20  3], shape=(64,), dtype=int32)


In [11]:
for i, (image_batch, labels_batch) in enumerate(ds_test2_preprocessed):
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  
  print('batch No {} with result: '.format(i), sum((class_pred == labels_batch).numpy()) / len(class_pred) * 100, '%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
batch No 0 with result:  85.9375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step
batch No 1 with result:  84.375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
batch No 2 with result:  89.0625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step  
batch No 3 with result:  82.75862068965517 %


2024-07-03 16:53:12.729321: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
